In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import gc
import os

In [2]:
import joblib
training = joblib.load('./model/training.pkl')

In [3]:
train_neg_sample = training.loc[training['click']==0]
train_neg_sample1  = train_neg_sample.sample(n=3000000*6)
train_pos_sample = training.loc[training['click']!=0]
train_pos_sample1 = train_pos_sample.sample(n=3000000)

In [4]:
training['click'].value_counts()

0        142981069
1          6359105
2           190699
3            43118
4            18570
5            10481
6             6773
7             4616
8             3297
9             2491
10            2125
11            1624
12            1390
13            1191
14             988
15             774
16             738
17             648
18             608
19             527
20             430
21             396
22             353
23             314
25             302
24             300
26             273
27             232
28             216
29             195
           ...    
514              1
512              1
511              1
510              1
505              1
578              1
491              1
488              1
218              1
483              1
480              1
478              1
528              1
530              1
532              1
534              1
536              1
537              1
540              1
542              1
543              1
549         

In [5]:
train = pd.concat([train_neg_sample1,train_pos_sample1]).reset_index(drop=True)
train['type']=0
train.dtypes
train

,click,impression,displayurl,adid,advertiserid,depth,position,queryid,keywordid,titleid,descriptionid,userid,type
0,0,1,7903914528320191889,21162526,1325,2,2,15,10,22,23,5413460,0
1,0,1,15145480155589094740,22072956,23807,2,1,1041676,269,3227,3495,2318239,0
2,0,1,17955036552515063201,20588302,33793,2,2,11494618,405,3626,306,2552325,0
3,0,1,7703279069701542953,20436320,4983,1,1,364,970,8738,5891,3310038,0
4,0,8,4892478347556874440,21729329,34211,3,3,96688,13130,512292,19161,0,0
5,0,2,12057878999086460853,20163238,27961,1,1,85963,169733,549966,467041,0,0
6,0,1,7484333488664585217,21090281,34691,2,2,2,67,78,103,2293353,0
7,0,1,2484495578050454714,21104688,33314,2,2,14199042,3349,16963,9351,11186497,0
8,0,1,13090221289647714786,20174985,23795,1,1,31,135,3,7,1694375,0
9,0,1,14340390157469404125,9027213,23808,3,2,18,1,0,0,2399266,0


In [6]:
num = train['impression'].sum()

In [7]:
def convert(train_array):
    convert_feature = np.zeros(shape=(num,13))
    index = 0
    for i in range(len(train_array)):
        row = train_array[i]
        #print(row)
        num_of_pos = int(row[0])
        for times in range(int(row[1])):
            if num_of_pos !=0:
                row[12] = int(1)
                convert_feature[index,:] = row
                num_of_pos = num_of_pos - 1
            else:
                row[12] = int(0)
                convert_feature[index,:] = row
            index = index + 1
    return convert_feature
train_array = train.values
train_data= convert(train_array)

In [8]:
import pandas as pd
columns = ['click','impression', 'displayurl', 'adid', 'advertiserid', 'depth', 
                     'position', 'queryid', 'keywordid', 'titleid', 'descriptionid', 'userid','type']
train_new = pd.DataFrame(train_data, columns=list(columns),dtype=np.int64)
train_new

,click,impression,displayurl,adid,advertiserid,depth,position,queryid,keywordid,titleid,descriptionid,userid,type
0,0,1,7903914528320191488,21162526,1325,2,2,15,10,22,23,5413460,0
1,0,1,-9223372036854775808,22072956,23807,2,1,1041676,269,3227,3495,2318239,0
2,0,1,-9223372036854775808,20588302,33793,2,2,11494618,405,3626,306,2552325,0
3,0,1,7703279069701542912,20436320,4983,1,1,364,970,8738,5891,3310038,0
4,0,8,4892478347556874240,21729329,34211,3,3,96688,13130,512292,19161,0,0
5,0,8,4892478347556874240,21729329,34211,3,3,96688,13130,512292,19161,0,0
6,0,8,4892478347556874240,21729329,34211,3,3,96688,13130,512292,19161,0,0
7,0,8,4892478347556874240,21729329,34211,3,3,96688,13130,512292,19161,0,0
8,0,8,4892478347556874240,21729329,34211,3,3,96688,13130,512292,19161,0,0
9,0,8,4892478347556874240,21729329,34211,3,3,96688,13130,512292,19161,0,0


In [9]:
train_new = train_new.drop_duplicates(subset=None, keep='first', inplace=False)

In [10]:
train_new['type'].value_counts()

0    18615464
1     2999992
Name: type, dtype: int64

In [11]:
import os
PATH_DATA = './model/'
test = None
if os.path.exists(PATH_DATA + 'test.pkl'):
    test = pd.read_pickle(PATH_DATA + 'test.pkl')
else:
    test = pd.read_csv(PATH_ABOVE + 'test.txt', header=None, sep='\t', nrows=None)
    test.columns = ['displayurl', 'adid', 'advertiserid', 'depth', 'position',
                    'queryid', 'keywordid', 'titleid', 'descriptionid', 'userid']
    test.to_pickle(PATH_DATA + 'test.pkl')
test = test[['adid', 'position', 'queryid', 'keywordid', 'userid']]
test

,adid,position,queryid,keywordid,userid
0,22166806,3,6415,34,490234
1,3111380,1,6415,8872,490234
2,22111129,2,6415,8872,490234
3,22166807,1,83,34,490234
4,20180629,2,83,34,490234
5,20163622,1,36,394,16870787
6,10400310,1,4595820,63,3514779
7,2580207,1,21942421,193,3514779
8,22219382,2,4595820,45245,3514779
9,20879032,1,12933754,3024,3514779


In [12]:
X = train_new[['adid', 'position', 'queryid', 'keywordid', 'userid']]
y = train_new[['click','impression','type']]

In [13]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

In [14]:
train_test=pd.concat([X,test])
train_test

,adid,position,queryid,keywordid,userid
0,21162526,2,15,10,5413460
1,22072956,1,1041676,269,2318239
2,20588302,2,11494618,405,2552325
3,20436320,1,364,970,3310038
4,21729329,3,96688,13130,0
12,20163238,1,85963,169733,0
14,21090281,2,2,67,2293353
15,21104688,2,14199042,3349,11186497
16,20174985,1,31,135,1694375
17,9027213,2,18,1,2399266


In [15]:
enc.fit(train_test)
train_onehot = enc.transform(X)
test_onehot = enc.transform(test)

/home/hujk/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [16]:
del train
del test
gc.collect()

145

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_onehot, y, test_size=0.1, random_state=42)

In [ ]:
from sklearn.svm import SVC
# clf = MultinomialNB(0.68)
# clf = SVC(kernel='linear')
# clf.fit(X_train, y_train['type'])
import os
from xgboost.sklearn import XGBClassifier
if os.path.exists('./model/xgb_train.pkl'):
    clf = joblib.load('./model/xgb_train.pkl')
else:
    clf = XGBClassifier(nthread=-1)
    clf.fit(X_train, y_train['type'])
    joblib.dump(clf, './model/xgb_train_9m.pkl')

In [ ]:
num_clicks = y_test['click'].tolist()

In [ ]:
num_impressions = y_test['impression'].tolist()

In [ ]:
prob_pre = clf.predict_proba(X_test)

In [ ]:
def scoreClickAUC(num_clicks, num_impressions, predicted_ctr):

    i_sorted = sorted(range(len(predicted_ctr)),key=lambda i: predicted_ctr[i],
                      reverse=True)
    auc_temp = 0.0
    click_sum = 0.0
    old_click_sum = 0.0
    no_click = 0.0
    no_click_sum = 0.0

    # treat all instances with the same predicted_ctr as coming from the
    # same bucket
    last_ctr = predicted_ctr[i_sorted[0]] + 1.0

    for i in range(len(predicted_ctr)):
        if last_ctr != predicted_ctr[i_sorted[i]]: 
            auc_temp += (click_sum+old_click_sum) * no_click / 2.0        
            old_click_sum = click_sum
            no_click = 0.0
            last_ctr = predicted_ctr[i_sorted[i]]
        no_click += num_impressions[i_sorted[i]] - num_clicks[i_sorted[i]]
        no_click_sum += num_impressions[i_sorted[i]] - num_clicks[i_sorted[i]]
        click_sum += num_clicks[i_sorted[i]]
    auc_temp += (click_sum+old_click_sum) * no_click / 2.0
    auc = auc_temp / (click_sum * no_click_sum)
    return auc

In [ ]:
auc = scoreClickAUC(num_clicks, num_impressions, prob_pre[:,1])
auc

In [ ]:
prob_pre_test = clf.predict_proba(test_onehot)

In [ ]:
predicted_ctr = prob_pre_test[:,1]

In [ ]:
solution = pd.read_csv('./data/KDD_Track2_solution.csv')

In [ ]:
auc = scoreClickAUC(solution['clicks'].tolist(), solution['impressions'].tolist(), predicted_ctr)
auc

In [ ]:
import joblib
joblib.dump(predicted_ctr,'./data/result/svc_firsttry.pkl')